In [1]:
import os
import sys
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [2]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig,  AutoTokenizer, AutoModelWithLMHead

In [3]:
torch.cuda.is_available() 

True

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [5]:
!pwd

/home/nmc/qg/Multi-Hop-QG/bart


In [7]:
# valid = pd.read_csv('../processed_dataset/hotpot_dev_distractor_supporting_facts_v1.csv')
valid = pd.read_csv('../processed_dataset/hotpot_dev_distractor_fullcontext_v1.csv')
valid = valid[2001:]

valid.head()

,Unnamed: 0,question,text
0,5a8b57f25542995d1e6f1371,Were Scott Derrickson and Ed Wood of the same ...,<answer> yes <context> Scott Derrickson (born ...
1,5a8c7595554299585d9e36b6,What government position was held by the woman...,<answer> Chief of Protocol <context> Kiss and ...
2,5a85ea095542994775f606a8,What science fantasy young adult series told i...,<answer> Animorphs <context> The Hork-Bajir Ch...
3,5adbf0a255429947ff17385a,Are the Laleli Mosque and Esma Sultan Mansion ...,<answer> no <context> The Laleli Mosque (Turki...
4,5a8e3ea95542995a26add48d,The director of the romantic comedy Big Stone ...,<answer> Greenwich Village New York City <cont...


In [8]:
PRETRAINED_MODEL = 'facebook/bart-base'
SEQ_LENGTH = 600
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)

tokenizer.add_special_tokens(
    {'additional_special_tokens': ['<answer>', '<context>']}
)

2

In [9]:
# md2 = torch.load("model_hotpot_supporting_facts_last.pth") # Supporting facts
md2 = torch.load("model_hotpot_full_context_last.pth") # Full context

In [10]:
def inference(review_text, model, device):
    encoded_text = tokenizer(
            review_text, 
            padding=True, 
            max_length=SEQ_LENGTH,
            truncation=True,
            return_tensors="pt"
        ).to(device)

    input_ids = encoded_text['input_ids']
    with torch.no_grad():
        output = model.generate(input_ids)
    decoded_string = tokenizer.decode(output[0], skip_special_tokens=True)
    return decoded_string

In [11]:
file_output = "predicted_supporting_facts.csv"
csvfile = open(file_output, 'w')
csvfile.write("question,predicted"+"\n")

for i in range(0, len(valid)):
    test_output= valid.iloc[i, 1]
    review_text = valid.iloc[i, 2]
#     print(review_text)
    predicted = inference(review_text, md2, device)
    line1 = test_output + "," + predicted
    csvfile.write(line1+"\n")
    
    if i%500 ==0:
        print(i)
csvfile.close()

/home/nmc/anaconda3/envs/questiongen/lib/python3.9/site-packages/transformers/generation_utils.py:1838: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000


In [ ]:
file_output = "predicted_full_context.csv"
csvfile = open(file_output, 'w')
csvfile.write("question,predicted"+"\n")

for i in range(0, len(valid)):
    test_output= valid.iloc[i, 1]
    review_text = valid.iloc[i, 2]
#     print(review_text)
    predicted = inference(review_text, md2, device)
    line1 = test_output + "," + predicted
    csvfile.write(line1+"\n")
    
    if i%500 ==0:
        print(i)
csvfile.close()